In [1]:
import sys, os
ROOT = os.path.abspath(os.curdir)
PARENT = os.path.dirname(ROOT)

if PARENT not in sys.path:
    sys.path.insert(0, PARENT)


In [2]:
import numpy as np
import scipy as sp
import sympy as sy
import torch
import copy
import time

import src.Simulator as sim_system
import src.Optimizer as opt
import src.SimGrad as sim_diff


ModuleNotFoundError: No module named 'scipy'

In [ ]:


###* Input files and function
reactions_file = "../reactions/reactionsSimpleV1.json"

const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

initial_state_dict = {'O_F': 0.1, 'O2_F':0.1 ,'O_S': 0.1, 'Vdb_S':0.1, 
                      'Odb_S': 0.1, 'CO_F': 0.1, 'CO2_F':0.1, 'CO_S': 0.1, 
                      'COdb_S': 0.0}

###* Functions for the data transformation
def compute_flux(const_dict, exp_dict, specie, molar_mass):
    den = exp_dict.get(specie, 0.0)
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux


def compute_remaining_flux(const_dict, exp_dict, molar_mass): 
    den = exp_dict['N'] - exp_dict['O'] - exp_dict['CO']
    v_th = np.sqrt((8.0 * const_dict['R'] * 1000 * exp_dict['Tnw'])/(molar_mass * np.pi))
    flux = 0.25 * v_th * den * 100
    return flux

####? EavgMB data extracted from the Booth et al. 2019 paper
p_data_exp = [0.2, 0.3, 0.4, 0.5, 0.6, 0.75, 1.5]
EavgMB_data = [1.04, 0.91, 0.87, 0.83, 0.77, 0.5, 0.001]
interpolator = sp.interpolate.interp1d(p_data_exp, EavgMB_data, kind='linear', fill_value=0.001, bounds_error=False)


transformations_exp = {
    'Tw':       lambda const_dict, exp_dict: exp_dict['Tw'] + 273.15,
    'fluxO' :   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O', 0.016),
    'fluxO2' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O2', 0.032),
    'fluxO3' :  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict,'O3', 0.048),
    'fluxC':    lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'C', 0.012),
    'fluxCO':   lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO', 0.028),
    'fluxCO2':  lambda const_dict, exp_dict: compute_flux(const_dict, exp_dict, 'CO2', 0.048),
    'EavgMB':   lambda const_dict, exp_dict: interpolator(exp_dict['pressure']).item(),
    'Ion':      lambda const_dict, exp_dict: 1e14 * exp_dict["current"]
}

output_folder_path = "../Buffer_Data"
exp_data_file = "Experimental_data_CO_Jorge.hdf5"


In [ ]:

exp_file = os.path.join(output_folder_path, exp_data_file)
sim = sim_system.Simulator(reactions_file, const_dict, exp_file, initial_state_dict, transformations_exp=transformations_exp)

  d[CO2_F]/dt = -CO2_F*r_29 + r_28*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_F*O_S*r_39 - CO_F*r_31 - CO_F*r_33 - 0.02*CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_30*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[CO_S]/dt = CO_F*r_35*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - CO_S*O_F*r_38 - CO_S*r_36 + r_32*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[O2_F]/dt = -O2_F*O_F*r_15 - O2_F*r_10 - O2_F*r_12 - O2_F*r_14 + r_9*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_F]/dt = -CO_F*O_F*r_34 - 0.02*CO_S*O_F*r_38 - O2_F*O_F*r_15 - 2*O_F**2*r_8 - 0.02*O_F*O_S*r_7 - 0.02*O_F*Odb_S*r_27 - 0.02*O_F*Vdb_S*r_26 - O_F*r_11 - O_F*r_2 - O_F*r_4 - 0.02*O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) + r_1*(-CO2_F - CO_F - O2_F - O_F + 1.0)
  d[O_S]/dt = -CO_F*O_S*r_39 - O_F*O_S*r_7 + O_F*r_5*(-CO_S - O_S - Odb_S - Vdb_S + 1.0) - O_S*r_16 - O_S*r_17 - O_S*r_37 - O_S*r_6 + r_3*(-CO_S - O_S - Odb_S - Vdb_S + 1.0)
  d[Odb_S]/dt = -O_F*Odb_S*r_27 + O_F*Vdb_S*r_26 - Odb_S*r_23 - Odb_S*r_24 - Odb

In [ ]:

def func_optimization(params, flag='numpy'):
    
    A_d, B_d, E_d, A_D, B_D, E_D, SF_1, SF_2, SF_3, SF_4 = params
    
    if flag=='numpy':
        nu_d_mod = lambda T: 1e15 * (B_d + A_d * np.exp(E_d/(const_dict['R'] * T)))
        nu_D_mod = lambda T: 1e13 * (B_D + A_D * np.exp(E_D/(const_dict['R'] * T)))
    elif flag=='torch':
        nu_d_mod = lambda T: 1e15 * (B_d + A_d * torch.exp(E_d/(const_dict['R'] * T)))
        nu_D_mod = lambda T: 1e13 * (B_D + A_D * torch.exp(E_D/(const_dict['R'] * T)))
    else:
        raise ValueError(f"{flag} does not exist")
    
    dict_mod_vec = [
    {"id": 2, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 10, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    {"id": 31, "rate": None, "model_dict": {"nu_d": nu_d_mod}},
    
    {"id": 5, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
    {"id": 7, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
    {"id": 8, "rate": None, "model_dict": {"nu_D": nu_D_mod}},
    
    {"id": 34, "rate": None, "model_dict": {"SF": SF_1}},
    {"id": 35, "rate": None, "model_dict": {"SF": SF_2}},
    {"id": 36, "rate": None, "model_dict": {"SF": SF_3}},
    {"id": 37, "rate": None, "model_dict": {"SF": SF_4}},
    ]
    
    return dict_mod_vec



def loss_function(exp, teo, flag='numpy'):
    
    func = ((teo-exp)**2)/(exp**2)
    if flag == 'numpy':
        return np.mean(func)
        
    elif flag == 'torch':
        return torch.mean(func)
    
    else:
        raise ValueError(f"{flag} does not exist")


# params = (1e-4, 1e-2, 1e-2, 1e-2)
params = (1e-4, 1e-5, 19.0, 1e-4, 1e-5, 19.0, 1e-1, 1e-2, 1e-1, 1e-2)


# optimizer = opt.Optimizer(sim, 
#                           lambda params: func_optimization(params, 'numpy'), 
#                           lambda exp, teo: loss_function(exp, teo, 'numpy')
#                           )

# diff = sim_diff.SimDiff(sim, 
#                     lambda params: func_optimization(params, 'torch'),
#                     params_default=params,
#                     gamma_exp_data=sim.gamma_exp_data_arr,
#                     loss_function=lambda exp, teo: loss_function(exp, teo, 'torch')
#                     )

In [ ]:
##! Input Space Discretization

def lhs_sampling(n_samples, n_dims):
    ###* Generate Latin Hypercube Samples in [0,1]^n_dims.
    
    samples = np.zeros((n_samples, n_dims))
    for j in range(n_dims):
        perm = np.random.permutation(n_samples)
        samples[:, j] = (perm + np.random.rand(n_samples)) / n_samples
    
    return samples


def map_to_bounds(samples, lower_bounds, upper_bounds):
    ###*  Linearly map samples in [0,1]^n_dims to [lower_bounds, upper_bounds].
    return lower_bounds + samples * (upper_bounds - lower_bounds)


lower_bounds = np.array([1e-8, 1e-8, 0.0, 1e-8, 1e-8, 0.0, 1e-5, 1e-5, 1e-5, 1e-5])
upper_bounds = np.array([1e-1, 1e-1, 30.0, 1e-1, 1e-1, 30.0, 1.0, 1.0, 1.0, 1.0])

n_samples = 10

lhs_samples = lhs_sampling(n_samples, len(lower_bounds))

samples = map_to_bounds(lhs_samples, lower_bounds, upper_bounds)
sample_zero = np.array(params)

print(samples.shape)


(10, 10)


In [ ]:
import multiprocessing

def f_and_grad(params, sim, params_default, lower_bounds, upper_bounds, samples):
    # Import inside function if issues persist
    import src.Optimizer as opt
    import src.SimGrad as sim_diff
    import torch
    import numpy as np
    import scipy as sp

    # Re-create objects inside each process (avoid globals)
    optimizer = opt.Optimizer(sim, 
                          lambda params: func_optimization(params, 'numpy'), 
                          lambda exp, teo: loss_function(exp, teo, 'numpy')
                          )
    diff = sim_diff.SimDiff(sim, 
                        lambda params: func_optimization(params, 'torch'),
                        params_default=params_default,
                        gamma_exp_data=sim.gamma_exp_data_arr,
                        loss_function=lambda exp, teo: loss_function(exp, teo, 'torch')
                        )

    print("params:", params)
    loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = optimizer.objective_function_diff(params)
    grad_val = diff.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
    print("loss_val: ", loss_val)
    return loss_val, grad_val.numpy()

def run_local_optimization(params_arr, sim, params_default, lower_bounds, upper_bounds, samples):
    import scipy as sp
    res = sp.optimize.minimize(
        lambda params: f_and_grad(params, sim, params_default, lower_bounds, upper_bounds, samples),
        x0=params_arr,
        jac=True,
        bounds=[(a_i, b_i) for a_i, b_i in zip(lower_bounds, upper_bounds)],
        method='L-BFGS-B')
    return res

if __name__ == '__main__':
    
    lower_bounds = np.array([1e-8, 1e-8, 0.0, 1e-8, 1e-8, 0.0, 1e-5, 1e-5, 1e-5, 1e-5])
    upper_bounds = np.array([1e-1, 1e-1, 30.0, 1e-1, 1e-1, 30.0, 1.0, 1.0, 1.0, 1.0])

    n_samples = 10

    lhs_samples = lhs_sampling(n_samples, len(lower_bounds))

    samples = map_to_bounds(lhs_samples, lower_bounds, upper_bounds)
    
    multiprocessing.set_start_method("spawn", force=True)

    # All heavy imports and setup go here!
    import src.Optimizer as opt
    import src.SimGrad as sim_diff
    import torch
    import numpy as np
    import scipy as sp
    # Your data/params setup here
    # sim, params_default, lower_bounds, upper_bounds, samples = ...

    num_workers = 2
    from pathos.multiprocessing import ProcessPool

    
    params_default = (1e-4, 1e-5, 19.0, 1e-4, 1e-5, 19.0, 1e-1, 1e-2, 1e-1, 1e-2)
    # Pass ALL needed data to worker (no globals!)
    results = []
    with ProcessPool(nodes=num_workers) as pool:
        results = pool.map(
            lambda params: run_local_optimization(params, sim, params_default, lower_bounds, upper_bounds, samples),
            samples
        )

    print(results)


In [ ]:

# import multiprocessing

# if __name__ == '__main__':
    
#     optimizer = opt.Optimizer(sim, 
#                           lambda params: func_optimization(params, 'numpy'), 
#                           lambda exp, teo: loss_function(exp, teo, 'numpy')
#                           )

#     diff = sim_diff.SimDiff(sim, 
#                         lambda params: func_optimization(params, 'torch'),
#                         params_default=params,
#                         gamma_exp_data=sim.gamma_exp_data_arr,
#                         loss_function=lambda exp, teo: loss_function(exp, teo, 'torch')
#                         )
    
    
#     def f_and_grad(params):
        
#         print("paranms:", params)
#         loss_val, frac_solutions_arr, rates_arr, _, gammas_predicted_arr = optimizer.objective_function_diff(params)
#         grad_val = diff.objective_function_grad(params, frac_solutions_arr, rates_arr, gammas_predicted_arr)
#         print("loss_val: ", loss_val)
#         return loss_val, grad_val.numpy()

#     def run_local_optimization(params_arr):
#         res = sp.optimize.minimize(f_and_grad,
#                     x0=params_arr,
#                     jac=True,
#                     bounds=[(a_i, b_i) for a_i,b_i in zip(lower_bounds, upper_bounds)],
#                     method='L-BFGS-B')
#         return res

    
    
#     num_workers = 2
#     multiprocessing.set_start_method("spawn", force=True)
#     from pathos.multiprocessing import ProcessPool
#     # import torch
#     pool = ProcessPool(nodes=num_workers)
#     results = pool.map(run_local_optimization, samples)
    
#     print(results)
    
    



In [ ]:

# best_res, best_loss = None, float('inf')
# f_calls = 0

# res_vec = []
# for i in range(n_samples+1):
    
#     if i == 0:
#         res = run_local_optimization(sample_zero)
        
#     else:    
#         res = run_local_optimization(samples[i-1])
    
#     res_vec.append(res)
#     print("res: ", res)
    
#     loss_opt = res.fun
#     f_calss = res.nfev
    
#     if loss_opt < best_loss:
#         best_theta, best_loss = res.x, loss_opt